In [1]:
"""
This file provides example code for how to perform postprocessing and analysis on volume data using the tomo3dt package.
"""

# Import code
import sys
import os 
import pickle
import numpy as np
import skimage
sys.path.append('../')

import tomo3dt
from tomo3dt import operators

In [8]:
# initial save path and time for collection of each 3D structure
path =  os.path.join(os.getcwd(),'..', "test_data")
sim_path = os.path.join(path,"simulation.vmf")
vol_path = os.path.join(path,"volume.vmf")
seg_path = os.path.join(path,"segmentation.vmf")

# Generate tilt series and load volumes for example 
times = np.linspace(1, 71, 71)
gr = (1+np.sqrt(5))/2
alpha = np.radians(140)
angles = np.degrees((times*gr*alpha % alpha)-(alpha/2)) 

sim = tomo3dt.VolumenDt(sim_path)
vol = tomo3dt.VolumenDt(vol_path)



In [ ]:
# perform otsu thresholding on every 3d structure in volume 
algor  = lambda x : x > skimage.filters.threshold_otsu(x)
segmented_vol = operators.segment(vol, algor, seg_path)

# Alternatively you can write over the preexisting volume file
segmented_vol = operators.segment(vol, algor)

In [10]:
# You can get quantification results by comparing a volume to a tilt series, volume or just off itself depending on the algorithm used 
# this extrapolates the metric to apply for every 3d structure in the volume
ssim_func = lambda x,y : skimage.metrics.structural_similarity(x,y,data_range=100)

# compare reconstructed to simulated volume
qr1 = operators.evaluate(ssim_func, vol, sim)

# compare forward projection of reconstructed volume to tilt series
ts = pickle.load(open(os.path.join(path,"tiltseries.pkl"), "rb"))
qr2 = operators.evaluate(ssim_func, vol, ts)

pickle.dump(qr1, open(os.path.join(path,"qr1.pkl"), "wb"))
pickle.dump(qr2, open(os.path.join(path,"qr2.pkl"), "wb"))

TypeError: fp() takes 2 positional arguments but 3 were given